In [3]:
import torch
import numpy as np
import sqlite3
from transformers import T5Tokenizer, T5ForConditionalGeneration

c:\Users\haris\miniconda3\envs\jazz\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# simplify the harmony

In [ ]:
# process harmonic data

In [ ]:
# process melodic data

In [ ]:
# data augmentation


In [5]:
# modelling
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [8]:
from preprocess import preprocess
from torch.utils.data import Dataset, DataLoader

In [9]:
class JazzDataset(Dataset):
    def __init__(self):
        super(JazzDataset).__init__()
        self.harmony, self.melody, self.har_to_i, self.i_to_har, self.mel_to_i, self.i_to_mel = preprocess()

    def __len__(self):
        return self.melody.shape[0]
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        melody = self.melody[index, :]
        harmony = self.harmony[index, :]
        sample = {"melody": melody, "harmony": harmony}
        
        return sample

In [10]:
# dress the data
BATCH_SIZE=32

dataset = JazzDataset()
dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)


# Determine the size of the training set (e.g., 80% of the data)
train_size = int(0.8 * len(dataloader.dataset))

# Determine the size of the validation set (e.g., 20% of the data)
valid_size = len(dataloader.dataset) - train_size

# Split the DataLoader into training and validation sets
train_dataset, valid_dataset = torch.utils.data.random_split(dataloader.dataset, [train_size, valid_size])

# Define the batch size for training
batch_size = 32

# Create the train_dataloader using the training dataset
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create the valid_dataloader using the validation dataset
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [12]:
# training
model.to(device)
model.train()
num_epochs = 5
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(num_epochs):
    running_loss=0
    for batch in train_dataloader:
        optimizer.zero_grad()
        harmony = batch['harmony'].long()
        melody = batch['melody'].long()

        attention_mask = torch.tril(torch.ones_like(melody)).to(device)
        
        outputs = model(input_ids=harmony, attention_mask=attention_mask, labels=melody)
        loss = outputs.loss
        running_loss += loss.item()
        print("h")
        loss.backward()
        optimizer.step()
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))


In [14]:
t = torch.tensor([1, 2, 3])
dict = {t: 1}
print(dict)
dict[torch.tensor([1, 2, 3])]

{tensor([1, 2, 3]): 1}


KeyError: tensor([1, 2, 3])

In [11]:
# testing
model.eval()
for batch in valid_dataloader:
    harmony = batch['harmony'].long()
    outputs = model.generate(harmony)
    print(outputs)
    print(batch['melody'])


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 16334884864 bytes.